In [27]:
import sys
import os

# Add the parent directory (project root) to sys.path
sys.path.append(os.path.abspath('..'))

from sandbox_steps.main import DesktopSteps
from e2b_desktop import Sandbox
E2B_API_KEY = os.environ.get("E2B_API_KEY")

In [28]:
# automatically load changes from the sandbox_steps module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
# With custom configuration
desktop = Sandbox(
    display=":0",  # Custom display (defaults to :0)
    resolution=(1920, 1080),  # Custom resolution
    timeout = 3600) 

desktop.stream.start()

# Get stream URL
url = desktop.stream.get_url()
print(url)

https://6080-igkwu2umozbiummh5luag-7f5cc07c.e2b.app/vnc.html?autoconnect=true&resize=scale


In [22]:
running_sandboxes = Sandbox.list()
running_sandboxes

[SandboxInfo(sandbox_id='irxv1iz3v4nnxqdjk943o-59d5f765', template_id='k0wmnzir0zuzye6dndlw', name='desktop', metadata={}, started_at=datetime.datetime(2025, 4, 29, 23, 5, 44, 437581, tzinfo=tzutc()), end_at=datetime.datetime(2025, 4, 30, 0, 5, 44, 437581, tzinfo=tzutc()))]

In [8]:
Sandbox.kill("ig67ifl7itvszhdnlxm3h-6cc9e88b")

True

In [ ]:
Sandbox.left_click(100, 100) 
Sandbox.right_click(200, 200)

In [ ]:
def install_csgo(desktop: Sandbox):
    desktop.open("https://www.cybersports.lt/setup/")
    desktop.commands.run("sudo dpkg --add-architecture i386")
    desktop.commands.run("sudo apt update")
    desktop.wait(100)
    desktop.commands.run("sudo apt install -y wine32")
    desktop.wait(100)
    desktop.commands.run("wine .") # this throws an error for fun
    desktop.open("https://drive.google.com/u/0/uc?id=1TIsvGACSrQOr1tgPaVpJebH375LjLIV6&export=download")
    desktop.wait(ms=4000)
    desktop.left_click(460, 305) #click the Download button
    desktop.wait(ms=5000) 
    desktop.commands.run("unzip Downloads/Windows7DefaultFonts.zip -d Downloads")
    desktop.commands.run("mv Downloads/Windows7DefaultFonts/* ~/.wine/drive_c/windows/Fonts/")
    desktop.commands.run("wine Downloads/Counter-Strike-1.6-original.exe")
    desktop.left_click(880, 1040) # open the terminal (I know, clean)
    desktop.wait(100)
    desktop.left_click(1800, 800) # open the terminal (I know, clean)
    desktop.write("wine Downloads/Counter-Strike-1.6-original.exe", chunk_size=50, delay_in_ms=25) 
    desktop.wait(2000)
    desktop.press("enter") # start the exe with wine
    desktop.wait(6000)
    desktop.press("enter") # start the installation
    desktop.wait(2000)
    desktop.press("enter") # one menu
    desktop.wait(2000)
    desktop.press("enter") # second menu
    desktop.wait(2000)
    desktop.press("enter") # final install button
    desktop.wait(55000) # installation process
    desktop.press("enter") # LAUNCH









    


In [109]:
desktop.commands.run("wine .") # this throws an error for fun


CommandExitException: Command exited with code 186 and error:
0070:err:explorer:initialize_display_settings Failed to query current display settings for L"\\\\.\\DISPLAY1".
0024:err:module:process_init failed to load L"Z:\\home\\user", error c00000ba


In [112]:
desktop.left_click(880, 1040) # open the terminal (I know, clean)
desktop.wait(100)
desktop.left_click(1800, 800) # open the terminal (I know, clean)
desktop.write("wine Downloads/Counter-Strike-1.6-original.exe", chunk_size=50, delay_in_ms=25) 
desktop.wait(2000)
desktop.press("enter") # start the exe with wine
desktop.wait(6000)
desktop.press("enter") # start the installation
desktop.wait(2000)
desktop.press("enter") # one menu
desktop.wait(2000)
desktop.press("enter") # second menu
desktop.wait(2000)
desktop.press("enter") # final install button
desktop.wait(55000) # installation process
desktop.press("enter") # LAUNCH







KeyboardInterrupt: 

In [ ]:
desktop.left_click(700, 800) # open the terminal (I know, clean)


CommandResult(stderr='', stdout='Desktop\nDocuments\nDownloads\nMusic\nPictures\nPublic\nTemplates\nVideos\n', exit_code=0, error='')

In [ ]:
desktop.left_click(460, 305) # click on the button to download fonts

In [106]:
install_csgo(desktop=desktop)

CommandExitException: Command exited with code 186 and error:
wine: created the configuration directory '/home/user/.wine'
0048:err:explorer:initialize_display_settings Failed to query current display settings for L"\\\\.\\DISPLAY1".
0048:err:ole:StdMarshalImpl_MarshalInterface Failed to create ifstub, hr 0x80004002
0048:err:ole:CoMarshalInterface Failed to marshal the interface {6d5140c1-7436-11ce-8034-00aa006009fa}, hr 0x80004002
0048:err:ole:apartment_get_local_server_stream Failed: 0x80004002
0040:err:winediag:nodrv_CreateWindow Application tried to create a window, but no driver could be loaded.
0040:err:winediag:nodrv_CreateWindow Make sure that your X server is running and that $DISPLAY is set correctly.
0040:err:setupapi:SetupDefaultQueueCallbackW copy error 1812 L"@wineusb.sys,-1" -> L"C:\\windows\\inf\\wineusb.inf"
0050:err:winediag:nodrv_CreateWindow Application tried to create a window, but no driver could be loaded.
0050:err:winediag:nodrv_CreateWindow Make sure that your 

In [103]:
desktop.kill()